# Phase 2 BESS Optimizer Test Notebook

This notebook provides a flexible testing and validation harness for the BESS optimization framework.

**Purpose:** Run single-pass optimization scenarios (no MPC/Meta-Opt) and validate results.

**Structure:**
1. Setup & Imports
2. Configuration
3. Run Scenario
4. Save Results
5. Validation Plots

## Prerequisites

Before running this notebook, ensure:

1. **Market Data** - ONE of the following options:
   - **Option A (Fastest)**: Preprocessed country parquet: `data/parquet/preprocessed/{country}.parquet`
   - Example: `de_lu.parquet`, `hu.parquet`, `at.parquet`
   - Generated by `py_script/data/generate_preprocessed_country_data.py`
   - 10-100x faster than loading Excel

   - **Option B (Submission)**: Phase 2 Excel file: `data/TechArena2025_Phase2_data.xlsx`
   - Official Huawei Phase 2 data workbook
   - The notebook will automatically load and extract country data
   - Matches submission requirements

2. **Configuration Files**: Must exist in `data/p2_config/`:
   - `solver_config.json` - Solver settings
   - `aging_config.json` - Degradation model parameters (Model II/III)
   - `afrr_ev_weights_config.json` - aFRR activation probabilities

3. **Solver**: At least one MILP solver installed:
   - CPLEX (best performance)
   - Gurobi (best performance)
   - CBC (good open-source option)
   - HiGHS (fast open-source)
   - GLPK (fallback)

**Note**: The notebook will automatically select the fastest available data source.

## 📦 1. Setup & Imports

In [ ]:
# Standard library imports
import sys
import json
import time
from pathlib import Path
from datetime import datetime

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Data processing
import pandas as pd
import numpy as np

# Optimization models
from py_script.core.optimizer import (
    BESSOptimizerModelI,
    BESSOptimizerModelII,
    BESSOptimizerModelIII
)

# Visualization utilities
from py_script.visualization.optimization_analysis import (
    extract_detailed_solution,
    plot_da_market_price_bid,
    plot_afrr_energy_market_price_bid,
    plot_capacity_markets_price_bid,
    plot_soc_and_power_bids
)

# Results export
from py_script.validation.results_exporter import save_optimization_results

# Aging analysis plots
from py_script.visualization.aging_analysis import (
    plot_stacked_cyclic_soc,
    plot_calendar_aging_curve,
    plot_aging_validation_suite
)

print("✅ All imports successful!")
print(f"Project root: {project_root}")

## ⚙️ 2. Configuration

Load configuration files and define scenario parameters.

In [ ]:
# ============================================================================
# Configuration Files
# ============================================================================

# Define configuration paths
config_dir = project_root / "data" / "p2_config"

# Load solver config
solver_config_path = config_dir / "solver_config.json"
with open(solver_config_path, 'r') as f:
    solver_config = json.load(f)
    print(f"✅ Loaded solver config: {solver_config_path}")

# Load aging config
aging_config_path = config_dir / "aging_config.json"
with open(aging_config_path, 'r') as f:
    aging_config = json.load(f)
    print(f"✅ Loaded aging config: {aging_config_path}")

# Load aFRR EV weights config (if using expected value weighting)
afrr_ev_config_path = config_dir / "afrr_ev_weights_config.json"
with open(afrr_ev_config_path, 'r') as f:
    afrr_ev_config = json.load(f)
    print(f"✅ Loaded aFRR EV config: {afrr_ev_config_path}")

print("\nConfiguration files loaded successfully!")

In [ ]:
# ============================================================================
# Scenario Parameters
# ============================================================================

# Test scenario configuration (MODIFY THESE AS NEEDED)
TEST_COUNTRY = "CH"              # Options: DE_LU, AT, CH, HU, CZ
TEST_C_RATE = 0.5                # Options: 0.25, 0.33, 0.5
TEST_ALPHA = 1.0                 # Degradation weight (for Model II/III)
TEST_TIME_HORIZON_HOURS = 24     # Time horizon in hours
TEST_START_STEP = 0              # Starting time step (15-min intervals)
TEST_MODEL = "III"               # Options: "I", "II", "III"
USE_EV_WEIGHTING = False         # Enable aFRR expected value weighting
MAX_AS_RATIO = 0.8               # Max ancillary service ratio (80%)
DAILY_CYCLE_LIMIT = None         # Daily cycle limit (Model I only, None = disabled)
REQUIRE_SEQUENTIAL_SEGMENT_ACTIVATION = True   # Enforce strict sequential segment filling (prevents parallel segment charging)
LIFO_EPSILON_KWH = 0           # LIFO tolerance (1-10 kWh, larger = faster solve)
MAX_SOC = 0.9                    # Max state of charge limit
MIN_SOC = 0.1                    # Min state of charge limit

# Display scenario summary
print("=" * 80)
print("📋 TEST SCENARIO CONFIGURATION")
print("=" * 80)
print(f"Model:              {TEST_MODEL}")
print(f"Country:            {TEST_COUNTRY}")
print(f"Time Horizon:       {TEST_TIME_HORIZON_HOURS} hours")
print(f"Start Step:         {TEST_START_STEP}")
print(f"C-Rate:             {TEST_C_RATE}")
print(f"Max AS Ratio:       {MAX_AS_RATIO * 100:.0f}%")
if TEST_MODEL in ["II", "III"]:
    print(f"Alpha (degradation):{TEST_ALPHA}")
    print(f"LIFO Epsilon:       {LIFO_EPSILON_KWH} kWh ({LIFO_EPSILON_KWH/447.2*100:.1f}% of segment capacity)")
    print(f"Sequential Activation:   {REQUIRE_SEQUENTIAL_SEGMENT_ACTIVATION} (False = faster solve)")
if TEST_MODEL == "I" and DAILY_CYCLE_LIMIT is not None:
    print(f"Daily Cycle Limit:  {DAILY_CYCLE_LIMIT}")
print(f"EV Weighting:       {USE_EV_WEIGHTING}")
print("=" * 80)

In [ ]:
# ============================================================================
# Load Market Data
# ============================================================================

# Import data loading utilities
from py_script.data.load_process_market_data import load_preprocessed_country_data

# Option 1: Try loading preprocessed country-specific parquet (FASTEST)
preprocessed_dir = project_root / "data" / "parquet" / "preprocessed"
preprocessed_path = preprocessed_dir / f"{TEST_COUNTRY.lower()}.parquet"

if preprocessed_path.exists():
    print(f"[FAST PATH] Loading preprocessed data: {preprocessed_path.name}")
    country_data = load_preprocessed_country_data(TEST_COUNTRY, data_dir=preprocessed_dir)
    print(f"[OK] Loaded {len(country_data)} time steps for {TEST_COUNTRY} (preprocessed)")
    
else:
    # Option 2: Load from Excel using optimizer's Phase 2 pipeline (SUBMISSION PATH)
    excel_path = project_root / "data" / "TechArena2025_Phase2_data.xlsx"
    
    if excel_path.exists():
        print(f"[SUBMISSION PATH] Loading from Excel: {excel_path.name}")
        print("   This matches Huawei submission requirements...")
        
        # Create temporary optimizer for data loading
        temp_opt = BESSOptimizerModelI()
        
        # Load using new Phase 2 Excel loader
        print("   Loading Phase 2 market tables from Excel...")
        full_data = temp_opt.load_and_preprocess_data(str(excel_path))
        
        # Extract country-specific data
        print(f"   Extracting country data for {TEST_COUNTRY}...")
        country_data = temp_opt.extract_country_data(full_data, TEST_COUNTRY)
        print(f"[OK] Loaded {len(country_data)} time steps for {TEST_COUNTRY} (Excel)")
    else:
        print("\nERROR: No data source found!")
        print("Please ensure ONE of the following exists:")
        print(f"  1. Preprocessed parquet (fast): {preprocessed_path}")
        print(f"  2. Phase 2 Excel (submission): {excel_path}")
        print("\nTo generate preprocessed files, run:")
        print("  python py_script/data/generate_preprocessed_country_data.py")
        raise FileNotFoundError(f"No data source available for {TEST_COUNTRY}")

# Extract time window
horizon_steps = TEST_TIME_HORIZON_HOURS * 4  # 15-min intervals
end_step = TEST_START_STEP + horizon_steps

if end_step > len(country_data):
    raise ValueError(f"Requested end step {end_step} exceeds available data {len(country_data)}")

data_slice = country_data.iloc[TEST_START_STEP:end_step].copy()
data_slice.reset_index(drop=True, inplace=True)

print(f"\n[OK] Extracted time window: steps {TEST_START_STEP} to {end_step} ({TEST_TIME_HORIZON_HOURS} hours)")
print(f"   Time steps: {len(data_slice)}")

# Display data summary
print(f"\nMarket Data Summary:")
print(f"   DA Price:    {data_slice['price_day_ahead'].min():.2f} - {data_slice['price_day_ahead'].max():.2f} EUR/MWh")
print(f"   FCR Price:   {data_slice['price_fcr'].min():.2f} - {data_slice['price_fcr'].max():.2f} EUR/MW")
print(f"   aFRR+ Price: {data_slice['price_afrr_pos'].min():.2f} - {data_slice['price_afrr_pos'].max():.2f} EUR/MW")
print(f"   aFRR- Price: {data_slice['price_afrr_neg'].min():.2f} - {data_slice['price_afrr_neg'].max():.2f} EUR/MW")
print(f"   aFRR Energy+ Price: {data_slice['price_afrr_energy_pos'].min():.2f} - {data_slice['price_afrr_energy_pos'].max():.2f} EUR/MWh")
print(f"   aFRR Energy- Price: {data_slice['price_afrr_energy_neg'].min():.2f} - {data_slice['price_afrr_energy_neg'].max():.2f} EUR/MWh")

## 🚀 3. Run Scenario

Execute the complete optimization workflow:
1. Instantiate optimizer
2. Build optimization model
3. Solve model
4. Extract solution

In [ ]:
# ============================================================================
# Step 1: Instantiate Optimizer
# ============================================================================

print("\n" + "=" * 80)
print("🔧 STEP 1: INITIALIZE OPTIMIZER")
print("=" * 80)

# Select and instantiate the appropriate model
if TEST_MODEL == "I":
    optimizer = BESSOptimizerModelI()
    print(f"✅ Initialized Model I (Base 4-market optimization)")
    
elif TEST_MODEL == "II":
    optimizer = BESSOptimizerModelII(
        alpha=TEST_ALPHA,
        require_sequential_segment_activation=REQUIRE_SEQUENTIAL_SEGMENT_ACTIVATION,
        use_afrr_ev_weighting=USE_EV_WEIGHTING
    )
    # Override LIFO epsilon from notebook parameter
    optimizer.degradation_params['lifo_epsilon_kwh'] = LIFO_EPSILON_KWH
    
    print(f"✅ Initialized Model II (Base + Cyclic Aging)")
    print(f"   Alpha: {TEST_ALPHA}")
    print(f"   LIFO Epsilon: {LIFO_EPSILON_KWH} kWh")
    print(f"   Sequential Activation: {REQUIRE_SEQUENTIAL_SEGMENT_ACTIVATION}")
    
elif TEST_MODEL == "III":
    optimizer = BESSOptimizerModelIII(
        alpha=TEST_ALPHA,
        require_sequential_segment_activation=REQUIRE_SEQUENTIAL_SEGMENT_ACTIVATION,
        use_afrr_ev_weighting=USE_EV_WEIGHTING
    )
    # Override LIFO epsilon from notebook parameter
    optimizer.degradation_params['lifo_epsilon_kwh'] = LIFO_EPSILON_KWH
    
    print(f"✅ Initialized Model III (Base + Cyclic + Calendar Aging)")
    print(f"   Alpha: {TEST_ALPHA}")
    print(f"   LIFO Epsilon: {LIFO_EPSILON_KWH} kWh")
    print(f"   Sequential Activation: {REQUIRE_SEQUENTIAL_SEGMENT_ACTIVATION}")
else:
    raise ValueError(f"Invalid model: {TEST_MODEL}. Choose 'I', 'II', or 'III'")

# Configure optimizer
optimizer.max_as_ratio = MAX_AS_RATIO
optimizer.battery_params['soc_min'] = MIN_SOC
optimizer.battery_params['soc_max'] = MAX_SOC

print(f"\n📋 Optimizer Configuration:")
print(f"   Battery Capacity: {optimizer.battery_params['capacity_kwh']} kWh")
print(f"   Round-trip Eff:   {optimizer.battery_params['efficiency'] * 100:.1f}%")
print(f"   Max AS Ratio:     {optimizer.max_as_ratio * 100:.0f}%")
print(f"   Max SOC:          {MAX_SOC * 100:.0f}%")
print(f"   Min SOC:          {MIN_SOC * 100:.0f}%")
print(f"   EV Weighting:     {USE_EV_WEIGHTING}")

In [ ]:
# ============================================================================
# Step 2: Build Optimization Model
# ============================================================================

print("\n" + "=" * 80)
print("🏗️ STEP 2: BUILD OPTIMIZATION MODEL")
print("=" * 80)

build_start = time.time()

# Build model with appropriate parameters
if TEST_MODEL == "I" and DAILY_CYCLE_LIMIT is not None:
    model = optimizer.build_optimization_model(
        data_slice,
        c_rate=TEST_C_RATE,
        daily_cycle_limit=DAILY_CYCLE_LIMIT
    )
    print(f"✅ Model I built with daily_cycle_limit={DAILY_CYCLE_LIMIT}")
else:
    model = optimizer.build_optimization_model(
        data_slice,
        c_rate=TEST_C_RATE
    )

build_time = time.time() - build_start

print(f"✅ Model built in {build_time:.2f} seconds")
print(f"\n📊 Model Statistics:")
print(f"   Variables:   {model.nvariables()}")
print(f"   Constraints: {model.nconstraints()}")
print(f"   Time Steps:  {len(data_slice)}")
print(f"   Blocks:      {len(data_slice) // 16}")  # 4-hour blocks

In [ ]:
# ============================================================================
# Step 3: Solve Model
# ============================================================================

print("\n" + "=" * 80)
print("⚡ STEP 3: SOLVE OPTIMIZATION MODEL")
print("=" * 80)

solve_start = time.time()

# Solve the model (auto-detect solver)
solved_model, solver_results = optimizer.solve_model(model)

solve_time = time.time() - solve_start

print(f"✅ Model solved in {solve_time:.2f} seconds")
print(f"\n📈 Solver Results:")
print(f"   Status:      {solver_results.solver.status}")
print(f"   Termination: {solver_results.solver.termination_condition}")
print(f"   Solver:      {solver_results.solver.name if hasattr(solver_results.solver, 'name') else 'unknown'}")

In [ ]:
# ============================================================================
# Step 4: Extract Solution
# ============================================================================

print("\n" + "=" * 80)
print("📦 STEP 4: EXTRACT SOLUTION")
print("=" * 80)

# Extract solution dictionary
solution_dict = optimizer.extract_solution(solved_model, solver_results)

print(f"✅ Solution extracted")
print(f"\n💰 Objective Value: {solution_dict['objective_value']:.2f} EUR")

# Display profit components if available
if 'profit_components' in solution_dict:
    print(f"\n📊 Profit Components:")
    pc = solution_dict['profit_components']
    for key, value in pc.items():
        print(f"   {key:30s}: {value:10.2f} EUR")

# Display degradation metrics if available (Model II/III)
if 'degradation_metrics' in solution_dict:
    print(f"\n🔋 Degradation Metrics:")
    dm = solution_dict['degradation_metrics']
    for key, value in dm.items():
        if isinstance(value, (int, float)):
            print(f"   {key:30s}: {value:10.4f}")

print(f"\n⏱️ Timing:")
print(f"   Build Time:  {build_time:.2f}s")
print(f"   Solve Time:  {solve_time:.2f}s")
print(f"   Total Time:  {build_time + solve_time:.2f}s")

In [ ]:
# Create solution DataFrame for visualization and export
solution_df = extract_detailed_solution(solution_dict, data_slice, TEST_TIME_HORIZON_HOURS)

print(f"\n📊 Solution DataFrame created: {solution_df.shape}")
print(f"\nColumns: {list(solution_df.columns)}")

# Display first few rows
print(f"\nFirst 5 rows:")
display(solution_df.head())

## 💾 4. Save Results

Save solution data and metrics using the `results_exporter` utility.

In [ ]:
# ============================================================================
# Prepare Summary Metrics
# ============================================================================

# Calculate revenue breakdown from solution_df
revenue_da = solution_df['revenue_da_eur'].sum() if 'revenue_da_eur' in solution_df.columns else 0
revenue_fcr = solution_df['revenue_fcr_eur'].sum() if 'revenue_fcr_eur' in solution_df.columns else 0
revenue_afrr_cap = solution_df['revenue_afrr_capacity_eur'].sum() if 'revenue_afrr_capacity_eur' in solution_df.columns else 0
revenue_afrr_energy = solution_df['revenue_afrr_energy_eur'].sum() if 'revenue_afrr_energy_eur' in solution_df.columns else 0
total_revenue = revenue_da + revenue_fcr + revenue_afrr_cap + revenue_afrr_energy

# Build summary metrics dictionary
summary_metrics = {
    'model': TEST_MODEL,
    'country': TEST_COUNTRY,
    'time_horizon_hours': TEST_TIME_HORIZON_HOURS,
    'start_step': TEST_START_STEP,
    'c_rate': TEST_C_RATE,
    'max_as_ratio': MAX_AS_RATIO,
    'use_ev_weighting': USE_EV_WEIGHTING,
    'total_profit_eur': solution_dict['objective_value'],
    'total_revenue_eur': total_revenue,
    'revenue_da_eur': revenue_da,
    'revenue_fcr_eur': revenue_fcr,
    'revenue_afrr_capacity_eur': revenue_afrr_cap,
    'revenue_afrr_energy_eur': revenue_afrr_energy,
    'solver_status': solution_dict['status'],
    'solver_name': solution_dict.get('solver', 'unknown'),
    'solve_time_sec': solve_time,
    'build_time_sec': build_time,
    'total_time_sec': build_time + solve_time,
    'n_variables': model.nvariables(),
    'n_constraints': model.nconstraints()
}

# Add model-specific parameters
if TEST_MODEL in ['II', 'III']:
    summary_metrics['alpha'] = TEST_ALPHA
    summary_metrics['lifo_epsilon_kwh'] = LIFO_EPSILON_KWH
    summary_metrics['require_sequential_segment_activation'] = REQUIRE_SEQUENTIAL_SEGMENT_ACTIVATION
if TEST_MODEL == 'I' and DAILY_CYCLE_LIMIT is not None:
    summary_metrics['daily_cycle_limit'] = DAILY_CYCLE_LIMIT

# Add degradation metrics if available
if 'degradation_metrics' in solution_dict:
    summary_metrics['degradation_metrics'] = solution_dict['degradation_metrics']

print("✅ Summary metrics prepared")
print(f"\nTotal Revenue: {total_revenue:.2f} EUR")
print(f"Total Profit:  {solution_dict['objective_value']:.2f} EUR")

In [ ]:
# ============================================================================
# Save Results to Disk
# ============================================================================
SAVE_MODEL = True

if SAVE_MODEL:
    # Generate descriptive run name
    run_name = f"notebook_test_model{TEST_MODEL}_{TEST_COUNTRY}_{TEST_TIME_HORIZON_HOURS}h"
    if TEST_MODEL in ['II', 'III']:
        run_name += f"_alpha{TEST_ALPHA}_eps{LIFO_EPSILON_KWH}"

    # Save using results_exporter
    output_directory = save_optimization_results(
        solution_df,
        summary_metrics,
        run_name,
        base_output_dir=str(project_root / "validation_results" / "optimizer_validation")
    )

    print("\n" + "=" * 80)
    print("💾 RESULTS SAVED SUCCESSFULLY")
    print("=" * 80)
    print(f"📁 Output directory: {output_directory}")
    print(f"   📊 solution_timeseries.csv")
    print(f"   📋 performance_summary.json")
    print(f"   📈 plots/ (subdirectory created)")
    print("=" * 80)
else:
    print("\n" + "=" * 80)
    print("⚠️ RESULTS NOT SAVED (SAVE_MODEL = False)")
    print("=" * 80)

## 📊 5. Validation Plots

Generate standard market participation plots and aging validation plots.

### 5.1 Standard Market Participation Plots

These plots show the battery's participation in each market.

In [ ]:
# Define plots directory
plots_dir = output_directory / "plots"
title_suffix = f"{TEST_COUNTRY} - {TEST_TIME_HORIZON_HOURS}h - Model {TEST_MODEL}"

print("Generating market participation plots...")
print("=" * 80)

In [ ]:
# Plot 1: Day-Ahead Market
print("\n[1/4] Day-Ahead Market...")
fig_da = plot_da_market_price_bid(solution_df, title_suffix=title_suffix, use_timestamp=False)
fig_da.write_html(str(plots_dir / "da_market_price_bid.html"))
fig_da.show()
print("   ✅ Saved: da_market_price_bid.html")

In [ ]:
# Plot 2: aFRR Energy Market
print("\n[2/4] aFRR Energy Market...")
fig_afrr_e = plot_afrr_energy_market_price_bid(solution_df, title_suffix=title_suffix, use_timestamp=False)
fig_afrr_e.write_html(str(plots_dir / "afrr_energy_market_price_bid.html"))
fig_afrr_e.show()
print("   ✅ Saved: afrr_energy_market_price_bid.html")

In [ ]:
# Plot 3: Capacity Markets (FCR + aFRR)
print("\n[3/4] Capacity Markets...")
fig_cap = plot_capacity_markets_price_bid(solution_df, title_suffix=title_suffix, use_timestamp=False)
fig_cap.write_html(str(plots_dir / "capacity_markets_price_bid.html"))
fig_cap.show()
print("   ✅ Saved: capacity_markets_price_bid.html")

In [ ]:
# Plot 4: SOC & Power Bids
print("\n[4/4] SOC & Power Bids...")
fig_soc = plot_soc_and_power_bids(solution_df, title_suffix=title_suffix, use_timestamp=False)
fig_soc.write_html(str(plots_dir / "soc_and_power_bids.html"))
fig_soc.show()
print("   ✅ Saved: soc_and_power_bids.html")

print("\n" + "=" * 80)
print("✅ All market participation plots generated!")
print("=" * 80)

### 5.2 Aging Validation Plots (Model II/III Only)

These plots validate the degradation model implementations:
- **Cyclic SOC**: Stacked area chart of 10 SOC segments (Model II/III)
- **Calendar Aging**: SOS2 piecewise-linear cost curve (Model III only)

In [ ]:
# Check if aging plots are applicable based on model
if TEST_MODEL in ['II', 'III']:
    print("\nGenerating aging validation plots...")
    print("=" * 80)
    
    # Plot 5: Stacked Cyclic SOC (Model II/III)
    if 'e_soc_j' in solution_dict and solution_dict['e_soc_j']:
        print("\n[5/6] Cyclic SOC Stacked Segments...")
        try:
            fig_cyclic = plot_stacked_cyclic_soc(
                solution_dict,
                title_suffix=title_suffix,
                save_path=str(plots_dir / "cyclic_soc_stacked.html")
            )
            fig_cyclic.show()
            print("   ✅ Saved: cyclic_soc_stacked.html")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    else:
        print("\n[5/6] Cyclic SOC plot skipped (no segment data)")
    
    # Plot 6: Calendar Aging Curve (Model III only)
    if TEST_MODEL == 'III' and 'c_cal_cost' in solution_dict and solution_dict['c_cal_cost']:
        print("\n[6/6] Calendar Aging Cost Curve...")
        try:
            fig_calendar = plot_calendar_aging_curve(
                solution_dict,
                aging_config=aging_config,
                title_suffix=title_suffix,
                save_path=str(plots_dir / "calendar_aging_curve.html")
            )
            fig_calendar.show()
            print("   ✅ Saved: calendar_aging_curve.html")
        except Exception as e:
            print(f"   ❌ Error: {e}")
    else:
        print("\n[6/6] Calendar aging plot skipped (Model III required)")
    
    print("\n" + "=" * 80)
    print("✅ All aging validation plots generated!")
    print("=" * 80)
else:
    print("\n" + "=" * 80)
    print(f"⚠️ Aging plots skipped (Model {TEST_MODEL} does not include degradation modeling)")
    print("   To generate aging plots, use TEST_MODEL = 'II' or 'III'")
    print("=" * 80)

---

## 🎉 Notebook Complete!

### What was accomplished:
1. ✅ Loaded configuration files (solver, aging, aFRR EV weights)
2. ✅ Loaded and sliced market data for test scenario
3. ✅ Instantiated optimizer (Model I/II/III)
4. ✅ Built optimization model
5. ✅ Solved optimization problem
6. ✅ Extracted solution dictionary and DataFrame
7. ✅ Saved results using `results_exporter`
8. ✅ Generated all validation plots

### Output location:
All results are saved in the output directory shown above.

### Next steps:
- Modify scenario parameters in Section 2 and re-run
- Compare different models (I vs II vs III)
- Test different countries, time horizons, or alpha values
- **Tune LIFO_EPSILON_KWH**: Try 1, 5, or 10 kWh to balance accuracy vs speed
- Use saved results for further analysis or comparison